In [1]:
import pandas as pd

In [2]:
pd.__version__

'2.2.3'

In [2]:
df_2023_01 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_2023_07 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-07.parquet')

In [5]:
len(df_2023_01.columns)

19

In [6]:
len(df_2023_07.columns)

19

In [8]:
import sklearn

Read the data for January. How many columns are there?

19

In [10]:
df_2023_01['duration'] = df_2023_01.tpep_dropoff_datetime - df_2023_01.tpep_pickup_datetime
df_2023_01['duration'] = df_2023_01['duration'].dt.total_seconds()/60

In [12]:
df_2023_01.duration.describe()

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64

What's the standard deviation of the trips duration in January?

42.59


In [15]:
df_2023_01_filtered = df_2023_01[(df_2023_01.duration >=1) & (df_2023_01.duration <= 60)]

In [19]:
len(df_2023_01_filtered) / len(df_2023_01)

0.9812202822125979

What fraction of the records left after you dropped the outliers?

98%


In [21]:
features = ['PULocationID', 'DOLocationID']
label = ['trip_distance']

In [22]:
from sklearn.feature_extraction import DictVectorizer


In [24]:
df_2023_01_filtered[features] = df_2023_01_filtered[features].astype(str)

/tmp/ipykernel_3049/106575989.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2023_01_filtered[features] = df_2023_01_filtered[features].astype(str)


In [26]:
train_dicts = df_2023_01_filtered[features + label].to_dict(orient = 'records')

In [27]:
dv = DictVectorizer()
x_train = dv.fit_transform(train_dicts)

In [28]:
x_train.shape

(3009173, 516)

What's the dimensionality of this matrix (number of columns)?

515（maybe 516 here）

In [37]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt


In [39]:
target = 'duration'
y_train = df_2023_01_filtered[target].values

lr = LinearRegression()
lr.fit(x_train, y_train)

y_pred = lr.predict(x_train)

mse = mean_squared_error(y_train, y_pred)

In [40]:
rmse = sqrt(mse)


In [41]:
rmse

7.658396898909143

What's the RMSE on train?

7.64 (maybe 7.66 here)

In [36]:
lr

58.65104306122118

In [42]:
df_2023_01[features] = df_2023_01[features].astype(str)
train_dicts_new = df_2023_01[features + label].to_dict(orient = 'records')

dv = DictVectorizer()
x_train_new = dv.fit_transform(train_dicts_new)

y_train_new = df_2023_01[target].values
y_pred_new  = lr.predict(x_train_new)
mse_new = mean_squared_error(y_train_new, y_pred_new)

rmse_new = sqrt(mse_new)
rmse_new

ValueError: X has 519 features, but LinearRegression is expecting 516 features as input.